<a href="https://colab.research.google.com/github/spps-supalerk/ChatBot/blob/main/SET_Scrapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Scrape Data
from bs4 import BeautifulSoup
import requests
import urllib.parse
import time
from datetime import datetime
import re

# Manipulate Data
import numpy as np
import pandas as pd

# Ignore Error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
base_url = 'https://www.set.or.th/set/commonslookup.do?language=th&country=TH&prefix={}'
prefix_list = ['NUMBER'] + list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

# record timestamp
scraped_time = datetime.now()
scraped_time_text = scraped_time.strftime('%Y-%m-%d %H:%M:%S')

result = []
log = []

for prefix in prefix_list:
    url = base_url.format(prefix)
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    rows = soup.select('#maincontent table.table-profile tr')[1:]
    
    for row in rows:
        td_list = row.find_all('td')
        symbol = td_list[0].text.strip() if td_list else ''
        name = td_list[1].text.strip() if len(td_list) >= 2 else ''
        market = td_list[2].text.strip() if len(td_list) >= 3 else ''
        
        result.append((symbol, name, market, url, scraped_time_text))
        
    print(f'{prefix}: {len(rows)}')
    
    time.sleep(0.2)

print(f'Total: {len(result)} entries')

end_time = datetime.now()
mins, seconds = divmod((end_time-scraped_time).total_seconds(), 60)
print(f'Elapsed time: {int(mins)} minutes {int(seconds)} seconds')

df = pd.DataFrame(result, columns=['symbol','name','market','url','scraped_time'])

# export to json
output_file = 'stock_symbols.json'
df.to_json(output_file, orient='records', force_ascii=False, indent=2)
print(f'Result file: {output_file}')

NUMBER: 3
A: 67
B: 51
C: 55
D: 17
E: 20
F: 16
G: 27
H: 11
I: 26
J: 15
K: 28
L: 19
M: 51
N: 27
O: 8
P: 59
Q: 7
R: 18
S: 119
T: 109
U: 20
V: 11
W: 17
X: 1
Y: 3
Z: 3
Total: 808 entries
Elapsed time: 0 minutes 14 seconds
Result file: stock_symbols.json


In [ ]:
# read stock symbol list
symbol_filepath = './stock_symbols.json'
symbol_df = pd.read_json(symbol_filepath)

symbol_df = symbol_df[['symbol','name','market']]

# timestamp
scraped_time = datetime.now()
scraped_time_text = scraped_time.strftime('%Y-%m-%d %H:%M:%S')
scraped_time_part = scraped_time.strftime('%Y%m%d_%H%M%S')

print(f'WEB SCRAPING START: {scraped_time_text}\n')

base_url = 'https://www.set.or.th/set/factsheet.do?symbol={}&ssoPageId=3&language=th&country=TH'

result = []
log = []
n_total = symbol_df.shape[0]

for i,row in symbol_df.iterrows():
    symbol = row['symbol']
    
    try:
        url = base_url.format(urllib.parse.quote(symbol))
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        container = soup.select_one('body > table').find_all('tr',recursive=False)[2].td.find_all('table', recursive=False)[1]

        # header section
        header_section = container.find_all('tr', recursive=False)[0].td.table

        # industry/sector
        tokens = header_section.tr.td.table.find_all('td')[0].text.strip().split('/')
        industry = tokens[0].strip() if tokens else ''
        sector = tokens[1].strip() if len(tokens) > 1 else ''

        # value section
        value_section = container.find_all('tr', recursive=False)[1]
        labels = [re.sub(r'\s', ' ', td.text.strip()) for td in value_section.find_all('tr')[0].find_all('td')]
        values = [re.sub(r'\s', ' ', td.text.strip()) for td in value_section.find_all('tr')[1].find_all('td')]
        
        result_row = [('symbol', symbol), ('industry', industry), ('sector', sector)]
        result_row += list(zip(labels, values))
        result_row += [('url', url), ('scraped_time', scraped_time_text)]
                 
        result.append(dict(result_row))
        print(f'{i+1:5}/{n_total} {(i+1)/n_total:5.0%} [{symbol}] Done')
                 
    except Exception as e:
        error = e.__doc__
        print(f'{i+1:5}/{n_total} {(i+1)/n_total:5.0%} [{symbol}] error: {error}')
        
    log_msg = (symbol, error)
    log.append(log_msg)
    
    time.sleep(0.1)

end_time = datetime.now()
mins, seconds = divmod((end_time-scraped_time).total_seconds(), 60)
print(f'Elapsed time: {int(mins)} minutes {int(seconds)} seconds')

result_df = pd.DataFrame(result)
df = symbol_df.merge(result_df)

# export to json
output_file = f'./SET_factsheet_{scraped_time_part}.json'
df.to_json(output_file, orient='records', force_ascii=False, indent=2)
print(f'Result file: {output_file}')

WEB SCRAPING START: 2020-12-15 07:57:07

    1/808    0% [2S] Done
    2/808    0% [3K-BAT] Done
    3/808    0% [7UP] Done
    4/808    0% [A] Done
    5/808    1% [A5] Done
    6/808    1% [AAV] Done
    7/808    1% [ABICO] Done
    8/808    1% [ABM] Done
    9/808    1% [ABPIF] Done
   10/808    1% [ACAP] Done
   11/808    1% [ACC] Done
   12/808    1% [ACE] Done
   13/808    2% [ACG] Done
   14/808    2% [ADB] Done
   15/808    2% [ADVANC] Done
   16/808    2% [AEC] Done
   17/808    2% [AEONTS] Done
   18/808    2% [AF] Done
   19/808    2% [AFC] Done
   20/808    2% [AGE] Done
   21/808    3% [AH] Done
   22/808    3% [AHC] Done
   23/808    3% [AI] Done
   24/808    3% [AIE] Done
   25/808    3% [AIMCG] Done
   26/808    3% [AIMIRT] Done
   27/808    3% [AIRA] Done
   28/808    3% [AIT] Done
   29/808    4% [AJ] Done
   30/808    4% [AJA] Done
   31/808    4% [AKP] Done
   32/808    4% [AKR] Done
   33/808    4% [ALL] Done
   34/808    4% [ALLA] Done
   35/808    4% [ALT] Done
 